In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from scipy import io
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import cv2
import os

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding="same", use_bias="false", input_shape=(28,28,1)),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.35),
    layers.Dense(26, activation="softmax")
])

model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels - 1
test_labels = test_labels - 1

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

checkpoint = ModelCheckpoint(filepath="checkpoints/epoch_{epoch:02d}.keras", save_weights_only=False, save_freq="epoch")

model.fit(train_images, train_labels, epochs=15,
          validation_data=(test_images, test_labels), callbacks=[checkpoint], batch_size=64)

# Best = 95.13% validation acc

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels - 1
test_labels = test_labels - 1

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

model = tf.keras.models.load_model("alpha_identifier.keras")
loss, acc = model.evaluate(test_images, test_labels, batch_size=64)
print("Accuracy: ", acc*100, " Loss: ", loss)

325/325 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.9513 - loss: 0.1702
Accuracy:  95.12500166893005  Loss:  0.17023393511772156


In [ ]:
img = test_images[143]
cv2.imshow("hdb", img)
cv2.waitKey(1000)
img = img.reshape((-1,28,28,1))
prediction = model.predict(img)
print(np.argmax(prediction), test_labels[143])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
0 [0]
